In [ ]:
pip install requests pandas tqdm


In [ ]:
import requests
import pandas as pd
import time

API_KEY = "df79b09e5285aa22866ff2d15b68e7de"

def get_movies(page):
    url = f"https://api.themoviedb.org/3/movie/popular?api_key={API_KEY}&page={page}"
    response = requests.get(url)
    return response.json()

data_list = []

for page in range(1, 31):  # ~600 movies
    print(f"Downloading page {page}...")
    data = get_movies(page)

    for movie in data["results"]:
        title = movie.get("title")
        overview = movie.get("overview")
        genre_ids = movie.get("genre_ids")

        if overview and genre_ids:
            genre = genre_ids[0]

            data_list.append({
                "Title": title,
                "Description": overview,
                "genre": genre
            })

    time.sleep(0.3)

df = pd.DataFrame(data_list)
# Get genre mapping from TMDB
url = f"https://api.themoviedb.org/3/genre/movie/list?api_key={API_KEY}&language=en-US"
genre_data = requests.get(url).json()
genre_map = {g["id"]: g["name"] for g in genre_data["genres"]}

# Create new column with string genre
df["Genre_Name"] = df["genre"].map(genre_map)
df.head()
df.to_csv("movies_dataset.csv", index=False)

print("✅ Dataset created")



✅ Dataset created


In [ ]:
"""from google.colab import files
files.download("movies_dataset.csv")"""


'from google.colab import files\nfiles.download("movies_dataset.csv")'

In [ ]:
df.head(10)

,Title,Description,genre,Genre_Name
0,Greenland 2: Migration,Having found the safety of the Greenland bunke...,12,Adventure
1,The Wrecking Crew,Estranged half-brothers Jonny and James reunit...,28,Action
2,The Housemaid,"Trying to escape her past, Millie Calloway acc...",9648,Mystery
3,Zootopia 2,After cracking the biggest case in Zootopia's ...,16,Animation
4,Oscar Shaw,"After retiring from the police force, a relent...",28,Action
5,The Shadow's Edge,Macau Police brings the tracking expert police...,28,Action
6,Murder at the Embassy,1934. Private detective Miranda Green investig...,9648,Mystery
7,96 Minutes,"Former bomb disposal expert, Song Kang-Ren, an...",28,Action
8,Anaconda,A group of friends facing mid-life crises head...,12,Adventure
9,Avatar: Fire and Ash,In the wake of the devastating war against the...,878,Science Fiction


# Text Preprocessing

In [ ]:
df.shape

(598, 4)

#Lowercasing

In [ ]:
df['Description']= df['Description'].str.lower()

In [ ]:
df['Description']

,Description
0,having found the safety of the greenland bunke...
1,estranged half-brothers jonny and james reunit...
2,"trying to escape her past, millie calloway acc..."
3,after cracking the biggest case in zootopia's ...
4,"after retiring from the police force, a relent..."
...,...
593,john henry returns to his hometown in hopes of...
594,rémi is a man trapped in a deteriorating marri...
595,"it's the 21st century, the oil wars have made ..."
596,"held in an l.a. interrogation room, verbal kin..."


In [ ]:
df

,Title,Description,genre,Genre_Name
0,Greenland 2: Migration,having found the safety of the greenland bunke...,12,Adventure
1,The Wrecking Crew,estranged half-brothers jonny and james reunit...,28,Action
2,The Housemaid,"trying to escape her past, millie calloway acc...",9648,Mystery
3,Zootopia 2,after cracking the biggest case in zootopia's ...,16,Animation
4,Oscar Shaw,"after retiring from the police force, a relent...",28,Action
...,...,...,...,...
593,Forsaken,john henry returns to his hometown in hopes of...,37,Western
594,Beau Pere,rémi is a man trapped in a deteriorating marri...,18,Drama
595,Battletruck,"it's the 21st century, the oil wars have made ...",878,Science Fiction
596,The Usual Suspects,"held in an l.a. interrogation room, verbal kin...",18,Drama


# Remove Punctuation

In [ ]:
df['Description'] = df['Description'].str.replace(r"[^a-zA-Z0-9]", " ", regex=True)
df['Description'] = df['Description'].str.replace(r"\s+", " ", regex=True).str.strip()


In [ ]:
df['Description']

,Description
0,having found the safety of the greenland bunke...
1,estranged half brothers jonny and james reunit...
2,trying to escape her past millie calloway acce...
3,after cracking the biggest case in zootopia s ...
4,after retiring from the police force a relentl...
...,...
593,john henry returns to his hometown in hopes of...
594,r mi is a man trapped in a deteriorating marri...
595,it s the 21st century the oil wars have made a...
596,held in an l a interrogation room verbal kint ...


In [ ]:
df

,Title,Description,genre,Genre_Name
0,Greenland 2: Migration,having found the safety of the greenland bunke...,12,Adventure
1,The Wrecking Crew,estranged half brothers jonny and james reunit...,28,Action
2,The Housemaid,trying to escape her past millie calloway acce...,9648,Mystery
3,Zootopia 2,after cracking the biggest case in zootopia s ...,16,Animation
4,Oscar Shaw,after retiring from the police force a relentl...,28,Action
...,...,...,...,...
593,Forsaken,john henry returns to his hometown in hopes of...,37,Western
594,Beau Pere,r mi is a man trapped in a deteriorating marri...,18,Drama
595,Battletruck,it s the 21st century the oil wars have made a...,878,Science Fiction
596,The Usual Suspects,held in an l a interrogation room verbal kint ...,18,Drama


# Spelling Correction

In [ ]:
!pip install textblob
!python -m textblob.download_corpora


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
"""from textblob import TextBlob
from tqdm import tqdm
tqdm.pandas()

def correct_spelling(text):
    return str(TextBlob(text).correct())

df["Description"] = df["Description"].progress_apply(correct_spelling)"""

'from textblob import TextBlob\nfrom tqdm import tqdm\ntqdm.pandas()\n\ndef correct_spelling(text):\n    return str(TextBlob(text).correct())\n\ndf["Description"] = df["Description"].progress_apply(correct_spelling)'

In [ ]:
df

,Title,Description,genre,Genre_Name
0,Greenland 2: Migration,having found the safety of the greenland bunke...,12,Adventure
1,The Wrecking Crew,estranged half brothers jonny and james reunit...,28,Action
2,The Housemaid,trying to escape her past millie calloway acce...,9648,Mystery
3,Zootopia 2,after cracking the biggest case in zootopia s ...,16,Animation
4,Oscar Shaw,after retiring from the police force a relentl...,28,Action
...,...,...,...,...
593,Forsaken,john henry returns to his hometown in hopes of...,37,Western
594,Beau Pere,r mi is a man trapped in a deteriorating marri...,18,Drama
595,Battletruck,it s the 21st century the oil wars have made a...,878,Science Fiction
596,The Usual Suspects,held in an l a interrogation room verbal kint ...,18,Drama


# Stop Words Removal

In [ ]:
!pip install nltk


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stop_words])

df['Description'] = df['Description'].apply(remove_stopwords)


In [ ]:
df

,Title,Description,genre,Genre_Name
0,Greenland 2: Migration,found safety greenland bunker comet clarke dec...,12,Adventure
1,The Wrecking Crew,estranged half brothers jonny james reunite fa...,28,Action
2,The Housemaid,trying escape past millie calloway accepts job...,9648,Mystery
3,Zootopia 2,cracking biggest case zootopia history rookie ...,16,Animation
4,Oscar Shaw,retiring police force relentless detective hau...,28,Action
...,...,...,...,...
593,Forsaken,john henry returns hometown hopes repairing re...,37,Western
594,Beau Pere,r mi man trapped deteriorating marriage wife u...,18,Drama
595,Battletruck,21st century oil wars made mess planet land ou...,878,Science Fiction
596,The Usual Suspects,held l interrogation room verbal kint attempts...,18,Drama


# Tokenization

In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def tokenize_clean(text):
    tokens = word_tokenize(text)  # tokenize
    return tokens

df['Description'] = df['Description'].apply(tokenize_clean)

In [ ]:
df

,Title,Description,genre,Genre_Name
0,Greenland 2: Migration,"[found, safety, greenland, bunker, comet, clar...",12,Adventure
1,The Wrecking Crew,"[estranged, half, brothers, jonny, james, reun...",28,Action
2,The Housemaid,"[trying, escape, past, millie, calloway, accep...",9648,Mystery
3,Zootopia 2,"[cracking, biggest, case, zootopia, history, r...",16,Animation
4,Oscar Shaw,"[retiring, police, force, relentless, detectiv...",28,Action
...,...,...,...,...
593,Forsaken,"[john, henry, returns, hometown, hopes, repair...",37,Western
594,Beau Pere,"[r, mi, man, trapped, deteriorating, marriage,...",18,Drama
595,Battletruck,"[21st, century, oil, wars, made, mess, planet,...",878,Science Fiction
596,The Usual Suspects,"[held, l, interrogation, room, verbal, kint, a...",18,Drama


# Lemmatization

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

True

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(t, pos='v') for t in tokens]

df['Description'] = df['Description'].apply(lemmatize_tokens)



In [ ]:
df

,Title,Description,genre,Genre_Name
0,Greenland 2: Migration,"[find, safety, greenland, bunker, comet, clark...",12,Adventure
1,The Wrecking Crew,"[estrange, half, brothers, jonny, jam, reunite...",28,Action
2,The Housemaid,"[try, escape, past, millie, calloway, accept, ...",9648,Mystery
3,Zootopia 2,"[crack, biggest, case, zootopia, history, rook...",16,Animation
4,Oscar Shaw,"[retire, police, force, relentless, detective,...",28,Action
...,...,...,...,...
593,Forsaken,"[john, henry, return, hometown, hop, repair, r...",37,Western
594,Beau Pere,"[r, mi, man, trap, deteriorate, marriage, wife...",18,Drama
595,Battletruck,"[21st, century, oil, war, make, mess, planet, ...",878,Science Fiction
596,The Usual Suspects,"[hold, l, interrogation, room, verbal, kint, a...",18,Drama


In [ ]:
df['Lemmatized'] = df['Description'].apply(lemmatize_tokens)

In [ ]:
df

,Title,Description,genre,Genre_Name,Lemmatized
0,Greenland 2: Migration,"[find, safety, greenland, bunker, comet, clark...",12,Adventure,"[find, safety, greenland, bunker, comet, clark..."
1,The Wrecking Crew,"[estrange, half, brothers, jonny, jam, reunite...",28,Action,"[estrange, half, brothers, jonny, jam, reunite..."
2,The Housemaid,"[try, escape, past, millie, calloway, accept, ...",9648,Mystery,"[try, escape, past, millie, calloway, accept, ..."
3,Zootopia 2,"[crack, biggest, case, zootopia, history, rook...",16,Animation,"[crack, biggest, case, zootopia, history, rook..."
4,Oscar Shaw,"[retire, police, force, relentless, detective,...",28,Action,"[retire, police, force, relentless, detective,..."
...,...,...,...,...,...
593,Forsaken,"[john, henry, return, hometown, hop, repair, r...",37,Western,"[john, henry, return, hometown, hop, repair, r..."
594,Beau Pere,"[r, mi, man, trap, deteriorate, marriage, wife...",18,Drama,"[r, mi, man, trap, deteriorate, marriage, wife..."
595,Battletruck,"[21st, century, oil, war, make, mess, planet, ...",878,Science Fiction,"[21st, century, oil, war, make, mess, planet, ..."
596,The Usual Suspects,"[hold, l, interrogation, room, verbal, kint, a...",18,Drama,"[hold, l, interrogation, room, verbal, kint, a..."


In [ ]:
# Join tokens back
# If your column is tokenized
df['Cleaned_Text'] = df['Lemmatized'].apply(lambda x: " ".join(x))


In [ ]:
df

,Title,Description,genre,Genre_Name,Lemmatized,Cleaned_Text
0,Greenland 2: Migration,"[find, safety, greenland, bunker, comet, clark...",12,Adventure,"[find, safety, greenland, bunker, comet, clark...",find safety greenland bunker comet clarke deci...
1,The Wrecking Crew,"[estrange, half, brothers, jonny, jam, reunite...",28,Action,"[estrange, half, brothers, jonny, jam, reunite...",estrange half brothers jonny jam reunite fathe...
2,The Housemaid,"[try, escape, past, millie, calloway, accept, ...",9648,Mystery,"[try, escape, past, millie, calloway, accept, ...",try escape past millie calloway accept job liv...
3,Zootopia 2,"[crack, biggest, case, zootopia, history, rook...",16,Animation,"[crack, biggest, case, zootopia, history, rook...",crack biggest case zootopia history rookie cop...
4,Oscar Shaw,"[retire, police, force, relentless, detective,...",28,Action,"[retire, police, force, relentless, detective,...",retire police force relentless detective haunt...
...,...,...,...,...,...,...
593,Forsaken,"[john, henry, return, hometown, hop, repair, r...",37,Western,"[john, henry, return, hometown, hop, repair, r...",john henry return hometown hop repair relation...
594,Beau Pere,"[r, mi, man, trap, deteriorate, marriage, wife...",18,Drama,"[r, mi, man, trap, deteriorate, marriage, wife...",r mi man trap deteriorate marriage wife unexpe...
595,Battletruck,"[21st, century, oil, war, make, mess, planet, ...",878,Science Fiction,"[21st, century, oil, war, make, mess, planet, ...",21st century oil war make mess planet land out...
596,The Usual Suspects,"[hold, l, interrogation, room, verbal, kint, a...",18,Drama,"[hold, l, interrogation, room, verbal, kint, a...",hold l interrogation room verbal kint attempt ...


# Mapping Genre_id to Genre_Name

In [ ]:
#Count unique IDs
num_ids = df['genre'].nunique()
print(f"Number of unique genre IDs: {num_ids}")


Number of unique genre IDs: 19


In [ ]:
unique_ids = df['genre'].unique()
print("Unique IDs in dataset:", unique_ids)


Unique IDs in dataset: [   12    28  9648    16   878    18    27    35 10749    53    14 10752
    36    80    37 10751    99 10770 10402]


In [ ]:
#Count how many movies per id
df['genre'].value_counts()


,count
genre,
28,147
18,89
27,68
12,51
878,43
16,41
35,33
53,24
10749,24


# Preparing Text for Classification

In [ ]:
X=df['Cleaned_Text']
y=df['Genre_Name']

# Convert Text to Numbers

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # top 5000 words
X_vect = vectorizer.fit_transform(X)

# Splitting the Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.2, random_state=42)


# Classifying

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

clf = LogisticRegression(max_iter=500)
clf.fit(X_train, y_train)



LogisticRegression(max_iter=500)

In [ ]:
y_pred = clf.predict(X_test)
y_pred

array(['Adventure', 'Action', 'Action', 'Action', 'Action', 'Horror',
       'Action', 'Drama', 'Action', 'Action', 'Action', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Action', 'Adventure',
       'Action', 'Action', 'Action', 'Action', 'Action', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Action', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Action', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Horror', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Action', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Drama', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Action', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Action', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Action', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Action', 'Action',
       'Action', 'Action', 'Action', 'Action', 'Action', 'Action',
       'Action', 'Drama', 'Action', 'Drama', 'Action', 'Ac

In [ ]:
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

         Action       0.26      1.00      0.42        28
      Adventure       1.00      0.33      0.50         6
      Animation       0.00      0.00      0.00         8
         Comedy       0.00      0.00      0.00         9
          Crime       0.00      0.00      0.00         3
          Drama       0.89      0.38      0.53        21
         Family       0.00      0.00      0.00         1
        Fantasy       0.00      0.00      0.00         5
        History       0.00      0.00      0.00         1
         Horror       1.00      0.20      0.33        15
          Music       0.00      0.00      0.00         1
        Mystery       0.00      0.00      0.00         2
        Romance       0.00      0.00      0.00         6
Science Fiction       0.00      0.00      0.00         7
       Thriller       0.00      0.00      0.00         5
            War       0.00      0.00      0.00         1
        Western       0.00    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd

df_results = pd.DataFrame({
    "Description": df.loc[y_test.index, "Cleaned_Text"],
    "Actual": y_test,
    "Predicted": y_pred
})

df_results.head(10)


,Description,Actual,Predicted
110,armies mass final battle decide fate world pow...,Adventure,Adventure
287,master thief must pull dangerous casino heist ...,Action,Action
563,keller dover face every parent worst nightmare...,Drama,Action
77,follow ruthless hitman narrowly escape fatal a...,Action,Action
181,matt dream picturesque italian wed shatter fia...,Romance,Action
284,obsess go viral videos ghost paranormal events...,Horror,Horror
10,cast clan young predator find unlikely ally da...,Action,Action
468,humbert humbert middle age british novelist ap...,Drama,Drama
78,lucy college student along friends spend vacat...,Horror,Action
411,weekend getaway seclude lakeside estate group ...,Horror,Action


In [ ]:
new_text = "A young hero must save the world from an evil villain"
# Preprocess: lowercase, remove special chars, tokenize, remove stopwords, lemmatize
new_text_clean = " ".join([lemmatizer.lemmatize(w) for w in new_text.lower().split() if w.isalpha() and w not in stop_words])

# Convert to vector
new_vect = vectorizer.transform([new_text_clean])

# Predict
pred_genre = clf.predict(new_vect)
print(pred_genre)


['Action']
